# Install the unofficial API for ChatGPT. You will need to restart the notebook after this cell

In [1]:
from llm.chatgpt import ChatGPT

chatgpt_model = 'gpt-3.5-turbo'

chatgpt = ChatGPT() # I sent the API key on Slack. create a file called openai_key.txt in the llm folder and put the key in it.
def chatgpt_call(prompt):
    return chatgpt.get_response(prompt)

chatgpt_call("Generate one random sentence.")

'The sun sets over the horizon, casting a vibrant display of colors across the sky.'

In [ ]:

from utils import *

from typing import Dict, List, Union, Tuple, Any, Set, Type, ClassVar, Literal, Callable

from copy import deepcopy

import json

import unittest

from type import Coordinate, Direction

from nodes import Node, Room, Character, Item, Player

from world import World



from game import *
from collections import defaultdict
from difflib import get_close_matches
import nltk
import copy
import re


nltk.download('wordnet')
lemmatizer = nltk.stem.WordNetLemmatizer()
lemmatizer.lemmatize("werewolf")

def get_lemma(s):
    s = s.strip()
    if s[:2]=='a ' or s[:2]=='A ':
        s = s[2:]
    elif s[:3]=='an ' or s[:3]=='An ':
        s = s[3:]
    elif s[:4]=='the ' or s[:4]=='The ':
        s = s[4:]
    lemma = lemmatizer.lemmatize(s)
    return lemma

def edit_distance(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

MOVE_PATTERN = r'^Move \{[a-z1-9 \(\)]+\} to \{[a-z1-9. \(\)]+\}$'
ADD_PATTERN = r'^Add \{[a-z1-9 \(\)]+\}$'
DELETE_PATTERN = r'^Delete \{[a-z1-9 \(\)]+\}$'
DISPLAY_PATTERN = r'^Display \{[a-z1-9._ \(\)]+\}$'
SET_ATTRIBUTES_PATTERN = r'^Set \{[a-z1-9._ \(\)]+\} to \{(True|False|AnyValue)\}$'


class GameEngine:
    def __init__(self, map:Dict, game_logics:Dict):
        self.map = map
        self.game_logics = game_logics
        self.world = World(configurations={"map_size":(1,len(self.map))})
        self.action_state = defaultdict(lambda : False)
        self.action_template = defaultdict(int)
        self.action_general_constraints = defaultdict(list)
        self.action_game_related_constraints = defaultdict(list)
        self.action_effects = defaultdict(list)
        self.keywords_to_game_logic_action_map = []
        self.obtainable_item_to_game_logic_action_map = defaultdict(list)
        self.locations = [] # list of Room
        self.items = [] # list of Item
        self.npcs = [] # list of Character
        self.locations_ptrs = {} # dict of Room
        self.items_ptrs = {} # dict of Item
        self.npcs_ptrs = {} # dict of Character
        self.num_of_actions = 0
        self.pos_cnt = 0
        
        # TODO: descriptions for nodes should be provided or generated
        for location in self.map:
            # originally location starts with 'at _____'
            print("add node:",location[3:]) 
            self.locations.append(Room(location[3:], "You are "+location))
            self.locations_ptrs[location[3:]]=self.locations[-1]
            for entity in self.map[location]:
                if entity['type']=='npc':
                    npc_name = get_lemma(entity['content'][4:]) # originally npc starts with 'has _____' in the map dict
                    print("add node:",npc_name)
                    self.npcs.append(Character(npc_name,"A "+entity['content'][4:]))
                    self.npcs_ptrs[npc_name]=self.npcs[-1]
                    self.world.add_node(self.npcs[-1],self.locations[-1])
                elif entity['type']=='item':
                    item_name = get_lemma(entity['content'][4:]) # originally item starts with 'has _____' in the map dict
                    print("add node:",item_name)
                    self.items.append(Item(item_name,"A "+entity['content'][4:]))
                    self.items_ptrs[item_name]=self.items[-1]
                    self.world.add_node(self.items[-1],self.locations[-1])
            print('location on map',(0, self.pos_cnt))
            self.world.add_room(self.locations[-1], (0, self.pos_cnt))
            self.pos_cnt += 1     #TODO: this version doesn't care about how rooms are arraged

        # TODO: Player goal and description should be provided or generated
        self.player = Player("Player", description="You are the player.", goal="")
        #self.items.append(Item("money","A "+"money"))
        #self.items_ptrs["money"]=self.items[-1]
        
        self.world.add_node(self.player, self.locations[0])
        self.world.add_node(self.items[-1],self.player)

        print(self.items_ptrs)
        print(self.npcs_ptrs)
        print(self.locations_ptrs)

    def copy_constructor(self,origin) -> None:
        print(origin.__dict__)
        for k in origin.__dict__:
            print(k)
            setattr(self,k,copy.deepcopy(getattr(origin,k)))

    def save(self, world_file_name, engine_file_name, indent=4) -> None:
        # save game world
        self.world.save(world_file_name, indent=indent)

        # save game engine
        game_engine_data = {}
        for key in self.__dict__.keys():
            if key in ['world', 'locations', 'items', 'npcs', 'locations_ptrs', 'items_ptrs', 'npcs_ptrs', 'player']:
                continue
            else:
                game_engine_data[key] = copy.deepcopy(getattr(self, key))

        with open(engine_file_name, 'w', encoding = 'utf8') as f:
            f.write(json.dumps(game_engine_data, ensure_ascii=False, indent = indent))

    def load(self, world_file_name, engine_file_name) -> None:
        # load
        self.world = World.load(world_file_name)
        replay_game_engine_data = json.load(open(engine_file_name, 'r', encoding = 'utf8'))
        for key in replay_game_engine_data:
            setattr(self, key, replay_game_engine_data[key])

    def register_action(self, action_template:str) -> bool:
        # gives an action a unique id, returns True if the action is successfully registered
        if self.action_template[action_template]>0:
            return False
        self.num_of_actions += 1
        self.action_template[action_template] = self.num_of_actions
        return True

    def add_game_related_constraint(self, action_template, constraint):
        pass
        #TODO: not very important for now

    def add_action_effect(self, action_template:str, effect:str) -> None:
        assert self.action_template[action_template]>=1 # action_template must be registered and has an id
        if re.match(ADD_PATTERN,effect) or re.match(MOVE_PATTERN,effect) or re.match(DELETE_PATTERN,effect) or re.match(DISPLAY_PATTERN,effect) or re.match(SET_ATTRIBUTES_PATTERN,effect):
            self.action_effects[action_template].append(effect.strip())

    def apply_action(self,action_template:str, parameters, parametric_fields_attributes,replay_game_engine:Union['GameEngine', None]=None) -> None:
        '''
        Execute an action, apply the effects to the game world
        replay_game_engine does not apply the effects, but just checks the conditions and adds the action to the action list.
        TODO: this function needs refactoring. use the Action class. Also remove the replay_game_engine parameter in the future. Modifications to replay_game_engine should be done in (Reference 1) (Search comment "Reference 1")
        '''
        assert self.action_template[action_template]>=1
        move_action = []
        set_action = []
        add_action = []
        display_action = []
        delete_action = []

        self.action_general_constraints[action_template] = copy.deepcopy(parametric_fields_attributes)

        
        '''
        Preprocessing：parametric_fields_attributes keys are replaced by the corresponding entity names
        '''
        # Example: parameters {'enum(object)': 'book;sign'}
        # Example: parametric_fields_attributes {'enum(object)': {'is_enum_object': 'True', 'is_readable': 'AnyValue', 'message': 'AnyValue'}}
        temp_parametric_fields_attributes = {}
        for param in parameters:
            parameters[param] = parameters[param].split(';')
            #print(parametric_fields_attributes)
            attr_list = parametric_fields_attributes[param]
            for entity in parameters[param]:
                temp_parametric_fields_attributes[entity] = copy.deepcopy(attr_list)
        parametric_fields_attributes = temp_parametric_fields_attributes

        # Example after processing: 
        # parameters {'enum(object)': ['book', 'sign']}
        # parametric_fields_attributes {'book': {'is_enum_object': 'True', 'is_readable': 'AnyValue', 'message': 'AnyValue'}, 'sign': {'is_enum_object': 'True', 'is_readable': 'AnyValue', 'message': 'AnyValue'}}

        '''
        Register attributes to the World
        TODO: is_enum_object and is_object should be merged, same for is_enum_npc and is_npc
        '''
        for entity in parametric_fields_attributes:
            if "is_enum_object" in parametric_fields_attributes[entity] or "is_object" in parametric_fields_attributes[entity]:
                for attribute in parametric_fields_attributes[entity]:
                    self.world.register_new_attribute(Item, attribute, str, 'AnyValue')  #note: the register_new_attribute method won't change an existing attribute
            elif "is_enum_npc" in parametric_fields_attributes[entity] or "is_npc" in parametric_fields_attributes[entity]:
                for attribute in parametric_fields_attributes[entity]:
                    self.world.register_new_attribute(Character, attribute, str, 'AnyValue')  #note: the register_new_attribute method won't change an existing attribute
            elif "is_enum_room" in parametric_fields_attributes[entity] or "is_room" in parametric_fields_attributes[entity]:
                for attribute in parametric_fields_attributes[entity]:
                    self.world.register_new_attribute(Room, attribute, str, 'AnyValue')  #note: the register_new_attribute method won't change an existing attribute
        

        '''
        The following code checks the conditions of the action and raises an exception if the conditions are not met.
        TODO: Use the Condition class.
        '''
        # parameter_alias is a dictionary that maps the names in the parameters to the real names of the entities in the game world. usually should be the same.
        parameter_alias = {} 
        #print(parameters)
        #print(parametric_fields_attributes)
        for entity in parameters:
            if 'npc' in entity:
                for t in parameters[entity]:
                    alias = get_alias(self.npcs_ptrs.keys(), t) # alias is the real name of the npc.
                    if edit_distance(alias,get_lemma(t))>0: 
                        # This section of the code should be executed sparingly! 
                        # We have added npcs in the constructor of the game engine. 
                        # The same npc should not be added twice.
                        # It seems that the only case where this section of the code is executed is when something from 'game_logic' is not added.
                        # Any other case, if happens, should be considered a bug (e.g. case issues causing edit distance to be larger than 0).
                        # TODO: Check if we can remove this section of the code, and add npcs and items from 'game_logic' in the constructor of the game engine.
                        alias = get_lemma(t)
                        self.npcs.append(Character(alias,"A "+t))
                        self.npcs_ptrs[alias]=self.npcs[-1]
                        print("add node:",alias)
                        self.world.add_node(self.npcs[-1],self.player.container)
                        if replay_game_engine:
                            replay_game_engine.npcs.append(Character(alias,"A "+t))
                            replay_game_engine.npcs_ptrs[alias]=replay_game_engine.npcs[-1]
                            replay_game_engine.world.add_node(replay_game_engine.npcs[-1],replay_game_engine.world.find_node(replay_game_engine.player.container.name))
                    for attr in parametric_fields_attributes[t]:
                        # check initial attibutes before the action
                        # TODO: this check should be done using the Action class and the Condition class
                        if parametric_fields_attributes[t][attr]!='AnyValue' and self.npcs_ptrs[alias]._get_attribute(attr)!='AnyValue' and parametric_fields_attributes[t][attr]!= self.npcs_ptrs[alias]._get_attribute(attr):
                            raise Exception("Failed in adding a new action which requires a npc with attributes different from its current value")
                        print("set attribute:",alias,t,attr,parametric_fields_attributes[t][attr])
                        SetNodeAttributeOperation(self.world,alias,attr,parametric_fields_attributes[t][attr]).apply() # TODO: This either does nothing or erases the attributes and sets them to 'AnyValue'. This doesn't seem to be the intended behavior.
                        if replay_game_engine:
                            SetNodeAttributeOperation(replay_game_engine.world,alias,attr,parametric_fields_attributes[t][attr]).apply()
                    parameter_alias[t] = alias
            if 'object' in entity:
                for t in parameters[entity]:
                    alias = get_alias(self.items_ptrs.keys(), t)
                    if edit_distance(alias,get_lemma(t))>0:
                        alias = get_lemma(t)
                        self.items.append(Item(alias,"A "+t))
                        self.items_ptrs[alias]=self.items[-1]
                        print("add node:",alias)
                        self.world.add_node(self.items[-1],self.player.container)
                        if replay_game_engine:
                            replay_game_engine.items.append(Item(alias,"A "+t))
                            replay_game_engine.items_ptrs[alias]=replay_game_engine.items[-1]
                            replay_game_engine.world.add_node(replay_game_engine.items[-1],replay_game_engine.world.find_node(replay_game_engine.player.container.name))
                    for attr in parametric_fields_attributes[t]:
                        # initial attibutes before the action
                        if parametric_fields_attributes[t][attr]!='AnyValue' and self.items_ptrs[alias]._get_attribute(attr)!='AnyValue' and parametric_fields_attributes[t][attr]!= self.items_ptrs[alias]._get_attribute(attr):
                            raise Exception("Failed in adding a new action which requires a item with attributes different from its current value")
                        print("set attribute:",alias,t,attr,parametric_fields_attributes[t][attr])
                        SetNodeAttributeOperation(self.world,alias,attr,parametric_fields_attributes[t][attr]).apply()
                        if replay_game_engine:
                            SetNodeAttributeOperation(replay_game_engine.world,alias,attr,parametric_fields_attributes[t][attr]).apply()
                    parameter_alias[t] = alias
            if 'room' in entity:
                for t in parameters[entity]:
                    alias = sorted([i for i in self.locations_ptrs.keys()],key = lambda x:edit_distance(get_lemma(t), x))[0]
                    if edit_distance(alias,get_lemma(t))>0:
                        raise Exception("Location not found error: Try to access a location that doesn't exist")
                    for attr in parametric_fields_attributes[t]:
                        # initial attibutes before the action
                        if parametric_fields_attributes[t][attr]!='AnyValue' and self.locations_ptrs[alias]._get_attribute(attr)!='AnyValue' and parametric_fields_attributes[t][attr]!= self.locations_ptrs[alias]._get_attribute(attr):
                            raise Exception("Failed in adding a new action which requires a room with attributes different from its current value")
                        print("set attribute:",alias,t,attr,parametric_fields_attributes[t][attr])
                        SetNodeAttributeOperation(self.world,alias,attr,parametric_fields_attributes[t][attr]).apply()
                        if replay_game_engine:
                            SetNodeAttributeOperation(replay_game_engine.world,alias,attr,parametric_fields_attributes[t][attr]).apply()
                    parameter_alias[t] = alias
        #print("alias",parameter_alias)

        '''
        The following code is used to apply the action to the game engine.
        TODO: Refactor this code to use the Action class.
        '''
        for effect in self.action_effects[action_template]:      
            print("Apply:",effect) 
            if re.match(MOVE_PATTERN,effect):
                param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                assert len(param)==2
                print(param)
                param_object, param_to = param[0][1:-1],param[1][1:-1] # Example: ('object1', 'room1')
                if param_object not in parameters:
                    raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                if param_to=='inventory':
                    param_to = 'Player'
                elif param_to== "environment":
                    param_to = self.player.container.name
                elif '.inventory' in param[1]:
                    param_to= param_to.replace('.inventory','')
                elif param_to in parameters:
                    param_to = parameters[param_to]
                    if isinstance(param_to, list):
                        param_to = param_to[0]
                for t in parameters[param_object]:
                    print("Moving",parameter_alias[t],param_to)
                    can_proceed_flag = False
                    temp = self.world.find_node(parameter_alias[t]) # we use this to check the container of the entity
                    while temp:
                        if temp==self.player.container:
                            can_proceed_flag = True
                            break
                        else:
                            temp = temp.container
                    if can_proceed_flag==False:
                        raise Exception("Execution Error: in %s. You are not allowed to move an entity if the entity is in a different room"%s)
                                      
                    MoveNodeOperation(self.world, parameter_alias[t], param_to).apply()
            elif re.match(ADD_PATTERN,effect):
                param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                assert len(param)==1
                param_object = param[0][1:-1]
                if param_object not in parameters:
                    raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                for t in parameters[param_object]:
                    print("Added",parameter_alias[t])
                    AddNodeOperation(self.world, parameter_alias[t], Item, "A "+t, self.player.container.name).apply()
            elif re.match(DELETE_PATTERN,effect):
                param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                assert len(param)==1
                param_object = param[0][1:-1]
                if param_object not in parameters:
                    raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                for t in parameters[param_object]:
                    print("Deleted",parameter_alias[t])
                    DeleteNodeOperation(self.world, parameter_alias[t]).apply()
            elif re.match(SET_ATTRIBUTES_PATTERN ,effect):
                param = re.findall(r'\{[a-zA-Z1-9._ \(\)]+\}', effect)
                assert len(param)==2
                attribute, value = param[0][1:-1], param[1][1:-1]
                assert '.' in attribute
                assert value in ['True','False','AnyValue']
                param_object = attribute.split('.')[0]
                attribute_name = attribute.split('.')[1]
                if param_object not in parameters:
                    raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                for t in parameters[param_object]:
                    print("Set Attribute",parameter_alias[t],attribute_name, value)
                    SetNodeAttributeOperation(self.world, parameter_alias[t], attribute_name, value).apply()
            elif re.match(DISPLAY_PATTERN ,effect):
                param = re.findall(r'\{[a-zA-Z1-9._ \(\)]+\}', effect)
                assert len(param)==1
                attribute = param[0][1:-1]
                param_object = attribute.split('.')[0]
                if param_object not in parameters and param_object not in ['inventory','environment']:
                    raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()]+['inventory','environment'])))
                print("Displaying Message--- Detial:",effect)
                #TODO implement DisplayMessageOperation and fill this
            else:
                if "Display " in effect:
                    raise Exception("Unexpected effect format for Display: \"%s\". Expect one of %s"%(effect,', '.join(['Display {'+k+'.message}' for k in parameters.keys()]+['Display {inventory}','Display {environment}'])))
                elif "Add " in effect:
                    raise Exception("Unexpected effect format for Add: \"%s\". Expected format is Add {XXX}"%(effect))
                elif "Delete " in effect:
                    raise Exception("Unexpected effect format for Delete: \"%s\". Expected format is Delete {XXX}"%(effect))
                elif "Move " in effect:
                    raise Exception("Unexpected effect format for Move: \"%s\". Expected format is Move {XXX} to {XXX}")
                elif "Set " in effect:
                    raise Exception("Unexpected effect format for Set: \"%s\". Expected format is Set {XXX.some_attribute} to {True/False}")
                else:
                    raise Exception("%s is not an allowed action"%(effect))

    # TODO: This function may not be needed. Refactor using the Condition class
    # TODO: Preconditions are checked in two places. location, item needed, and attribute needed are checked here. field attribute values are checked in apply_action. Should be unified.
    def check_if_precondtions_have_been_met(self,action):
        '''
        check if certain preconditions has been met here before apply the action
        refers to game_logics
        e.g. get a sword at armory requires at armory
        assert self.player.container == self.world.find_node("armory")
        # TODO: return false if not met. Do not raise exception
        '''
        if self.world.find_node(self.game_logics[action]['location'][0][3:]) != self.world.player.container:
            raise Exception("You must be at %s to perform the action"%self.game_logics[action]['location'][0][3:])

        for item in self.game_logics[action]['item needed']:
            if self.world.find_node(get_lemma(item[4:])).container != self.world.player:
                raise Exception("You must obtain %s before perform the action"%item[4:])

        for event in self.game_logics[action]['preceeding_events']:
            if self.action_state[event]==False:
                raise Exception("You must finish [Quest]:%s before accepting this quest"%event)

        return True

    def check_if_disired_effects_were_applied(self,action):
        # TODO: return false if not met. Do not raise exception
        for disired_effect in self.game_logics[action]['results']:
            if disired_effect[:4]=='has ':
                if self.world.find_node(get_lemma(disired_effect[4:])).container != self.player:
                    raise Exception("Wrong effect error: %s should move object %s to inventory"%(action,disired_effect[4:]))
            else:
                # TODO: check if other effects are applied
                pass
        return True


[nltk_data] Downloading package wordnet to /home/chunhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:

from os import initgroups
import re
import copy
import random
from string import Formatter
import time
from typing import Match


# TODO: instruct GPT to only output attributes check and effect
action_generation_prompt = '''generate continuation of the following text.

Using the template provided define admissible action of a game engine. You are allowed to use only the following 5 action templates for action effects in exact format:
1. Move {entity} to {entity}  //Move an item 
2. Set {entity.some_attribute} to {True/False}  //set some_attributes of an entity
3. Delete {entity}  //remove material consumed by the player
4. Add {entity}  //Create items through crafting or alchemy. Item being created must not have existed before
5. Display {entity.message}  //entity is either an npc, location or item

Determine what attributes about the object should hold, and the effect of the action.

Here are some examples.

//action template for "get"
get {object1}:
attributes check: //pass
effect: Move {object1} to {inventory}

//action template for "eat"
eat {enum(object)}:
attributes check: {enum(object).is_edible==True}
effect: Delete {enum(object)}

//action template for "buy with"
buy {object1} with {object2}:
attributes check: {object1.is_for_sale==True}; {object2.is_currency==True};
effect: Move {object1} to {inventory}; Move {object2} to {environment}

//action template for "open"
open {object1}:
attributes check: {object1.is_open==False}; {object1.is_container==True}; {object1.is_locked==False}
effect: Set {object1.is_open} to {True}; Set {object1.is_locked} to {False}

//action template for "search"
search {location1}:
attributes check: //pass
effect: Display {location1.message}

//action template for "craft with"
craft {object1} with {enum(object)}:
attributes check: {enum(object).is_material==True};
npcs involved: //pass
effect: Add {object1}; Delete {enum(object)}

//action template for "hit with"
hit {npc1} with {object1}:
attributes check: {object1.is_weapon==True}; {npc1.is_alive==True};
effect: Set {npc1.is_alive} to {False}

//action template for "find"
find {object1}:
attributes check: //pass
effect: Move {object1} to {inventory}; Display {object1.message}

//action template for "listen"
listen to {npc1}:
attributes check: {npc1.is_alive==True}
effect: Display {npc1.message}

END of examples

Now, generate your response for the following action "{$text1$}".{$text4$}

//action template for "{$text1$}"{$text3$}
{$text2$}:
'''

find_enum_npc_prompt = '''
generate continuation of the following text.

find all npc/npcs appeared in the given sentence fragment.

Example 1:
Sentence fragment: farmers
npc/npcs: farmers

Example 2:
Sentence fragment: princess and wolf.
npc/npcs: princess, wolf

Example 3:
Sentence fragment: donkey.
npc/npcs: donkey

Example 4:
Sentence fragment: {$text1$}
npc/npcs:'''

find_enum_object_prompt = '''
generate continuation of the following text.

find all object/objects appeared in the given sentence fragment.

Example 1:
Sentence fragment: stones
object/objects: stones

Example 2:
Sentence fragment: wood, nail and steel.
object/objects: wood, nail, steel

Example 3:
Sentence fragment: pan and stove.
object/objects: pan, stove

Example 4:
Sentence fragment: {$text1$}
object/objects:'''

# TODO: what if no more npc can be found? what if no more object can be found? same for items
get_npc_that_fits_the_profile_prompt = '''generate continuation of the following text.

Given the sentence, infer the name of the npc involved in the action according to the given constraints

Example 1:
Sentence: adventurer drink holy water at church.
constraints: is_npc=True, is_alive==True
npc: priest

Example 2:
Sentence: adventurer buy armor at market place.
constraints: is_npc=True, is_vendor==True, is_alive==True
npc: armor vendor

Example 3:
Sentence: adventurer steal salt at kitchen.
constraints: is_npc=True, is_alive==True
npc: cook

Example 4:
Sentence: {$text1$}.
constraints: {$text2$}
npc:'''

action_template_extraction_prompt = '''generate continuation of the following text.

Identify action template for action in the sentences. Use {object1}, {object2} to replace objects in the action. Use {enum(object)} to represent a list of objects. Use {npc1}, {npc2} to replace npcs in the action. Use {enum(npc)} to represent a list of objects. Use {room1} to represent the location.

Example 1:
Sentence: climbed [object: wall] with [object: ladder].
Action template: climbed {object1} with {object2}

Example 2:
Sentence: stole [object: bow] from [room: blacksmith's workshop].
Action template: stole {object1} from {room1}

Example 3:
Sentence: crafted [object: shield] with [object: iron], [object: nail] and [object: wood].
Action template: crafted {object1} with {enum(object)}

Example 4:
Sentence: go to [room: farm].
Action template: go to {room1}

Example 5:
Sentence: purchase [object: gem] with [money].
Action template: purchase {object1} with {object2}

Example 6:
Sentence: {$text2$}
Action template:'''

expand_sentence_with_with_prompt = '''Generate continuation of the following text.

Determine if the preposition "with" can connect object with the sentence stem, if it is possible, connect the sentence stem with potential objects using "with". Otherwise answer "No". Here are some examples. Return the simplest form of the sentence.

sentence stem: adventurer craft sword.
potential object in the sentence: iron, wood, anvil
Answer: adventurer craft sword with iron, wood.

sentence stem: adventurer acquires a book 
potential object in the sentence: broom
Answer: No

sentence stem: {$text1$}
potential object in the sentence: {$text2$}
Answer:'''

fix_grammar_prompt = '''Generate continuation of the following text.

Fix all grammar mistakes in the following sentence.

Setence: adventurer kill a dragon at dragon's canyon.
Output: adventurer killed a dragon at dragon's canyon.

Setence: {$text1$}
Output:'''

extract_entities_prompt = '''Generate continuation of the following text.

Extract entities (object, creature, person, and locations) in the following sentence. Encapsulate entities with "[]".

Setence: The adventurer kill a dragon at dragon's canyon.
Output: [The adventurer] kill [a dragon] at [dragon's canyon].

Setence: {$text1$}
Output:'''

def fix_grammar(sentence) -> str:
    return chatgpt_call(fix_grammar_prompt.replace('{$text1$}',sentence)).split('\n')[0].split('.')[0].split('(')[0]

def extract_entities(sentence) -> str:
    '''
    Input: The adventurer kill a dragon at dragon's canyon.
    Output: [The adventurer] kill [a dragon] at [dragon's canyon]
    '''
    return chatgpt_call(extract_entities_prompt.replace('{$text1$}',sentence)).split('\n')[0].split('.')[0].split('(')[0]

def get_alias(candidates, word_to_alias) -> str:
    '''
    Get a candidate that is closest to word_to_alias
    '''
    return sorted(candidates,key = lambda x:edit_distance(get_lemma(word_to_alias), x))[0]

def fix_grammar_and_annotate_entities(sentence, all_objects, all_rooms, all_npcs) -> Tuple[str, str]:
    '''
    Input: adventurer kill a dragon at dragon's canyon.
    Output: [npc: the adventurer] killed [npc: a dragon] at [room: dragon's canyon]
    TODO: Concatenating characters not efficient.
    '''
    sentence_fixed_grammar = fix_grammar(sentence)
    annotated_sentence = extract_entities(sentence_fixed_grammar) # example: [The adventurer] killed [a dragon] at [dragon's canyon].

    start_of_entity = -1
    annotated_sentence_enhanced = ""
    for i in range(len(annotated_sentence)):
        if annotated_sentence[i]=='[':
            start_of_entity=i+1
        elif annotated_sentence[i]==']':
            prefix = ""
            entity = annotated_sentence[start_of_entity:i].strip().lower() # example: the adventurer

            # remove a, the, A, The
            for to_replace  in ['a ','the ','A ','The ']:
                if entity[:len(to_replace)]==to_replace:
                    entity = entity[len(to_replace):]
            if entity in all_objects:
                prefix = 'object: '
            elif entity in all_rooms:
                prefix = 'room: '
            elif entity in all_npcs:
                prefix = 'npc: '
            annotated_sentence_enhanced = annotated_sentence_enhanced+'['+prefix+entity+']'
            start_of_entity = -1
        elif start_of_entity==-1:
            annotated_sentence_enhanced = annotated_sentence_enhanced+annotated_sentence[i]


    print("original sentence:",sentence)
    print("fixed grammar:",sentence_fixed_grammar)
    print("annotate_sentence:",annotated_sentence_enhanced)
    return sentence_fixed_grammar, annotated_sentence_enhanced

def expand_sentence_with_with(sentence, objects) -> str:
    '''
    Example
    Sentence: adventurer craft sword.
    Objects: iron, wood, anvil
    Output: adventurer craft sword with iron, wood
    '''
    if objects==[]:
        return sentence
    else:
        result= chatgpt_call(expand_sentence_with_with_prompt.replace('{$text1$}',sentence).replace('{$text2$}',', '.join(objects))).split('\n')[0].split('.')[0].split('(')[0].lower()
        print(result)
        if result.strip() not in ["no","No"] and 'adventurer' in result: # TODO: Player may not be adventurer
            return result
        else:
            return sentence
        
def identify_action_template_given_the_sentence(sentence) -> str:
    '''
    Input: crafted [object: shield] with [object: iron], [object: nail] and [object: wood].
    Output: crafted {object1} with {enum(object)}
    '''
    prompt = action_template_extraction_prompt.replace("{$text2$}",' '.join(sentence.split(' ')[1:]))
    return chatgpt_call(prompt).split('\n')[0]

def get_npc_that_fits_the_profile(sentence,attributes) -> str:
    '''
    Sentence: adventurer buy armor at market place.
    attributes: is_npc=True, is_vendor==True, is_alive==True
    Output: armor vendor
    '''
    prompt = get_npc_that_fits_the_profile_prompt.replace('{$text1$}',sentence).replace('{$text2$}',', '.join([k+'='+attributes[k] for k in attributes]))
    infered_npc = chatgpt_call(prompt).split('\n\n')[0]
    #print(infered_npc)
    # TODO: infered_npc may be empty, or there may be multiple NPCs
    return infered_npc

def parse_enum_objects_npcs(sentence_fragment,type="object") -> List[str]:
    '''
    Sentence fragment: wood, nail and steel.
    Output: ['wood', 'nail', 'steel']
    '''
    if type=="object":
        prompt = find_enum_object_prompt.replace('{$text1$}',sentence_fragment)
    else:
        prompt = find_enum_npc_prompt.replace('{$text1$}',sentence_fragment)
    result = chatgpt_call(prompt).split('\n')[0].split('(')[0].replace(' and ',', ').split(',')
    result = [s.strip() for s in result if s.strip()!='']
    return result

def parsing_parameters(sentence, template) -> Dict[str,str]:
    '''
    Find parameter of action.
    sentence = adventurer craft sword with iron, wood and anvil.
    template = adventurer craft {object1} with {enum(object)}.
    Output: {'object1':'sword', 'enum(object)':'iron;wood;anvil'}
    '''
    fieldnames_in_template = [name for _, name, _, _ in Formatter().parse(template) if name]
    print(fieldnames_in_template)
    p = str(template)
    for fieldname in fieldnames_in_template:
        p = p.replace('{'+fieldname+'}','(.*)')
    print("matching pattern:",sentence,p)
    p = re.compile(p)
    result = p.search(sentence)
    # The following result type check should be implemented for good code style. However, since the null pointer exception in result.group is checked when adding action, having this code would break the game. 
    # if result is None:
    #     raise ValueError(f"Cannot parse \"{sentence}\" with \"{template}\"")
    parameters = {}
    for i,fieldname in enumerate(fieldnames_in_template):
        if fieldname=='enum(npc)':
            parameters[fieldname] = ';'.join(parse_enum_objects_npcs(result.group(i+1),type='npc'))
        elif fieldname=='enum(object)':
            parameters[fieldname] = ';'.join(parse_enum_objects_npcs(result.group(i+1),type='object'))
        elif fieldname[:5]=='enum(':
            raise ValueError('enum object can only be enum(object) or enum(npc).')
        else:
            parameters[fieldname] = result.group(i+1).replace('.','')
    print("parameters",parameters)
    return parameters

def get_action_template_and_rules(sentence,previous_attept_info = "") -> Tuple[str,str,str]:
    '''
    TODO: Check doc
    Input: adventure drink [object: the magic water] with [object: a bottle]
    Output: action_template, action_rule, attempt_info
    action_template = 'drink {object1} with {object2}'
    action_rule = 'attributes check: {object1.is_drinkable==True}; {object2.is_container==True} effect: Delete {object1}; Display {object1.message}'
    attempt_info = 
    '//action template for "drink with"
    drink {object1} with {object2}:
    attributes check: {object1.is_drinkable==True}; {object2.is_container==True}
    effect: Delete {object1}; Display {object1.message}'
    '''
    # TODO: Player may not be adventurer. Also, this is too hacky.
    for keyword in ['The adventurer','the adventurer','A adventurer', 'a adventurer', 'Adventurer', 'An adventurer', 'an adventurer']:
        if keyword in sentence:
            sentence = sentence.replace(keyword,'adventurer')
    sucess = False
    template = identify_action_template_given_the_sentence(sentence) # Example: crafted {object1} with {enum(object)}
    print(template)
    state = 0
    preprocessed = ""
    for i in range(len(template)):
        if template[i]=='{':
            state = 1
        elif template[i]=='}':
            state = 0
        else:
            if state==0:
                preprocessed = preprocessed+template[i]
    preprocessed = ' '.join([i for i in preprocessed.split(' ') if i!='']) # Example: crafted with
    prompt = action_generation_prompt.replace("{$text1$}",preprocessed).replace("{$text2$}",template).replace("{$text3$}",'\n//'+sentence).replace("{$text4$}",previous_attept_info)
    if previous_attept_info!="":
        print(prompt)
    action_rule = chatgpt_call(prompt).split('\n\n')[0]
    print(action_rule)
    return template,action_rule,prompt.split('\n\n')[-1]+action_rule

def try_adding_action(game_engine:GameEngine, original_sentence, sentence,sentence_expanded,room,template,rule,replay_game_engine=None) -> GameEngine:
    '''
    Try applying the action to the game engine.
    Return the new game engine if the action is successful, otherwise an exception is raised. 
    The original game engine is not modified.
    replay_game_engine should not to be modified, but when there are nodes in game_logic that are not added, they will be added to replay_game_engine.
    TODO: If check_if_disired_effects_were_applied is refactored, this function must be refactored as well.
    TODO: remove dependency on replay_game_engine. Modifications on replay_game_engine should be completely done at (Reference 1) (search comment "Reference 1").
    '''
    parameters = parsing_parameters(sentence, template) #find all parameters in the sentence  # Example: {'object1':'sword', 'enum(object)':'iron;wood;anvil'}
    # for each parameter in action, find their attributes
    parametric_fields_attributes = {} # The requirements for an action. Example: {'object1':{'is_object':'True','is_weapon':'True'}}
    for key in parameters:
        p = re.compile("\{%s\.([a-z,A-Z,_]+)"%key.replace('(','\(').replace(')','\)'))
        result = p.findall(rule)
        parametric_fields_attributes[key] = {}
        if key[:6]=='object':
            parametric_fields_attributes[key]['is_object']='True'
        elif key[:3]=='npc':
            parametric_fields_attributes[key]['is_npc']='True'
        elif key[:4]=='room':
            parametric_fields_attributes[key]['is_room']='True'
        elif key=='enum(npc)':
            parametric_fields_attributes[key]['is_enum_npc']='True'
        elif key=='enum(object)':
            parametric_fields_attributes[key]['is_enum_object']='True'
        for attr in result:
            p = re.compile("\{%s\.%s==[a-z,A-Z]+"%(key,attr))
            initial_state_of_attributes = list(set(p.findall(rule.split('\n')[0])))
            if len(initial_state_of_attributes)==0:
                parametric_fields_attributes[key][attr]='AnyValue'
            elif len(initial_state_of_attributes)==1:
                parametric_fields_attributes[key][attr]=initial_state_of_attributes[0].split('=')[-1]
            else:
                raise ValueError('parametric_fields_attributes cannot be True and False at the same time.')
    print("parametric_fields_attributes",parametric_fields_attributes)

    # non parametric fields. Some objects/npcs may be hidden/implicitly defined, 
    # this makes generation too slow, so removed from the this version
    '''
    non_parametric_fields = list(set([name for _, name, _, _ in Formatter().parse(rule) if name and '.' not in name and name not in fieldnames_in_template]))
    print(non_parametric_fields)
    non_parametric_fields_attributes = {}
    for field in non_parametric_fields:
        p = re.compile("\{%s\.([a-z,A-Z,_]+)"%field)
        result = p.findall(rule)
        non_parametric_fields_attributes[field] = {}
        if field[:6]=='object':
            non_parametric_fields_attributes[field]['is_object']='True'
        elif field[:3]=='npc':
            non_parametric_fields_attributes[field]['is_npc']='True'
        elif field[:4]=='room':
            non_parametric_fields_attributes[field]['is_room']='True'
        for attr in result:
            p = re.compile("\{%s\.%s==[a-z,A-Z]+"%(field,attr))
            initial_state_of_attributes = list(set(p.findall(rule.split('\n')[0])))
            if len(initial_state_of_attributes)==0:
                non_parametric_fields_attributes[field][attr]='AnyValue'
            elif len(initial_state_of_attributes)==1:
                non_parametric_fields_attributes[field][attr]=initial_state_of_attributes[0].split('=')[-1]
            else:
                raise ValueError('non_parametric_fields_attributes cannot be True and False at the same time.')
        if 'is_npc' in non_parametric_fields_attributes[field] and non_parametric_fields_attributes[field]['is_npc']=='True':
            non_parametric_fields_attributes[get_npc_that_fits_the_profile(sentence_expanded,non_parametric_fields_attributes[field])]=copy.deepcopy(non_parametric_fields_attributes[field])
            del non_parametric_fields_attributes[field]


    print("non_parametric_fields_attributes",non_parametric_fields_attributes)
    '''
    
    # parse the effects
    effects = [effect.strip() for effect in rule.split('\n')[-1][len('effect: '):].split(';')]
    print("effects:",effects)

    #input example for game_engine
    #template = 'gathered {enum(object)} for {object1} with {object2}'
    #parameters = {'enum(object)': 'ingredients', 'object1': 'a cure', 'object2': 'a basket'}
    #parametric_fields_attributes = {'enum(object)': {'is_enum_object': 'True'}, 'object1': {'is_object': 'True', 'is_cure': 'True', 'is_complete': 'AnyValue'}, 'object2': {'is_object': 'True', 'is_container': 'True'}}
    #effects = ['Move {enum(object)} to {object2}', 'Set {object1.is_complete} to {True}']

    game_engine_copy = copy.deepcopy(game_engine)     
    assert game_engine_copy.world.player is not None
    #since we omit the walk to action in the walk through
    # omit location requirement
    # TODO: SHOULD BE REMOVED IN THE FUTURE
    if game_engine_copy.world.find_node(game_engine_copy.game_logics[original_sentence]['location'][0][3:]) != game_engine_copy.world.player.container: 
        MoveNodeOperation(game_engine_copy.world, 'Player', game_engine_copy.game_logics[original_sentence]['location'][0][3:]).apply() # TODO: player may not be called player
    
    game_engine_copy.check_if_precondtions_have_been_met(original_sentence)
    if game_engine_copy.register_action(template)==True:
        print("register new action")
        for effect in effects:
            game_engine_copy.add_action_effect(template,effect)
    else:
        print("use existing action")
    game_engine_copy.apply_action(template, parameters, parametric_fields_attributes,replay_game_engine)
    game_engine_copy.check_if_disired_effects_were_applied(original_sentence)
    return game_engine_copy
    

In [ ]:

game =  {'game_logics': {'adventurer speak with the village elders.': {'item needed': [], 'location': ['at village hall'], 'preceeding_events': [], 'description': [], 'results': []}, 'adventurer read books.': {'item needed': [], 'location': ['at library'], 'preceeding_events': [], 'description': ['There is a book at the library.'], 'results': ['has book']}, 'adventurer find maps.': {'item needed': ['has books'], 'location': ['at library'], 'preceeding_events': [], 'description': ['There is a map at the library.'], 'results': ['has map']}, 'adventurer take torch.': {'item needed': ['has money'], 'location': ['at general store'], 'preceeding_events': [], 'description': ['There is a torch for sale at the general'], 'results': ['has torch']}, 'adventurer find the werewolf den.': {'item needed': ['has maps', 'has torch'], 'location': ['at forest'], 'preceeding_events': ['adventurer speak with the village elders.'], 'description': [], 'results': []}, 'adventurer take basket.': {'item needed': [], 'location': ['at forest'], 'preceeding_events': [], 'description': ['There is a basket in the forest.'], 'results': ['has basket']}, 'adventurer gather ingredients for a cure.': {'item needed': ['has basket'], 'location': ['at forest'], 'preceeding_events': ['adventurer find the werewolf den.'], 'description': [], 'results': []}, 'adventure meet with a wise woman in the forest': {'item needed': [], 'location': ['at forest'], 'preceeding_events': ['adventurer gather ingredients for a cure.'], 'description': [], 'results': []}, 'adventurer purchase flashlight.': {'item needed': ['has money'], 'location': ['at store'], 'preceeding_events': [], 'description': ['There is a flashlight for sale at the store'], 'results': ['has flashlight']}, 'adventurer investigate the abandoned mansion': {'item needed': ['has flashlight'], 'location': ['at abandoned mansion'], 'preceeding_events': ['adventure meet with a wise woman in the forest'], 'description': [], 'results': []}, 'adventurer purchase lockpicking tool.': {'item needed': ['has money'], 'location': ['at general store'], 'preceeding_events': [], 'description': ['There is a lockpicking tool for sale at'], 'results': ['has lockpicking tool']}, 'adventurer steal key.': {'item needed': ['has lockpicking tool'], 'location': ['at thieves den'], 'preceeding_events': [], 'description': ['There is a key at thieves den.'], 'results': ['has key']}, 'adventurer take sword.': {'item needed': ['has key'], 'location': ['at weapon depot'], 'preceeding_events': [], 'description': ['There is a sword at a weapon depot.'], 'results': ['has sword']}, 'adventure confront the werewolf pack leader': {'item needed': ['has sword'], 'location': ['at werewolf den'], 'preceeding_events': ['adventurer investigate the abandoned mansion'], 'description': [], 'results': []}, 'adventurer loot weapons.': {'item needed': [], 'location': ['at dungeon'], 'preceeding_events': [], 'description': ['There is a loot of weapons at the d'], 'results': ['has weapons']}, 'adventurer find armor.': {'item needed': ['has money'], 'location': ['at armor shop'], 'preceeding_events': [], 'description': ['There is armor for sale at an armor'], 'results': ['has armor']}, 'adventure escort villagers to safety': {'item needed': ['has weapons', 'has armor'], 'location': ['at village'], 'preceeding_events': ['adventure confront the werewolf pack leader'], 'description': [], 'results': []}, 'adventure destroy the cursed artifact': {'item needed': [], 'location': ['at abandoned mansion'], 'preceeding_events': ['adventure escort villagers to safety'], 'description': [], 'results': []}, 'adventurer stop the curse from spreading to other villages': {'item needed': [], 'location': ['at forest'], 'preceeding_events': ['adventure destroy the cursed artifact'], 'description': [], 'results': []}}, 'map': {'at village hall': [{'type': 'npc', 'content': 'has village elders'}], 'at forest': [{'type': 'npc', 'content': 'has werewolf'}, {'type': 'npc', 'content': 'has farmers'}, {'type': 'npc', 'content': 'has herbalist'}, {'type': 'npc', 'content': 'has witch'}, {'type': 'item', 'content': 'has basket'}, {'type': 'npc', 'content': 'has wise woman'}], 'at general store': [{'type': 'item', 'content': 'has torch'}, {'type': 'item', 'content': 'has lockpicking tool'}, {'type': 'npc', 'content': 'has merchant'}], 'at library': [{'type': 'item', 'content': 'has maps'}, {'type': 'item', 'content': 'has books'}], 'at abandoned mansion': [], 'at store': [{'type': 'item', 'content': 'has flashlight'}, {'type': 'npc', 'content': 'has merchant'}], 'at werewolf den': [{'type': 'npc', 'content': 'has werewolf pack leader'}], 'at weapon depot': [{'type': 'item', 'content': 'has sword'}], 'at thieves den': [{'type': 'item', 'content': 'has key'}], 'at village': [{'type': 'npc', 'content': 'has villagers'}], 'at armor shop': [{'type': 'item', 'content': 'has armor'}], 'at dungeon': [{'type': 'item', 'content': 'has weapons'}]}}

action_mapping = {key:expand_sentence_with_with(key,game['game_logics'][key]['item needed']) for key in game['game_logics']}

   

no
no
adventurer find the werewolf den with maps, torch
adventurer gather ingredients for a cure with a basket
no
no
no
no
no
adventure confront the werewolf pack leader with sword
no
no


In [ ]:
'''
The following code reads game logics, map, and generates the story. It also looks for all objects and rooms in the game.
'''
game_logics = game['game_logics']
# the following code changes the actions game_logics to the expanded version. 
# e.g. adventure confront the werewolf pack leader is changed to adventure confront the werewolf pack leader with sword
# game['game_logics']['preceeding_events'] is also updated to reflect the changes.
# Other fields in game['game_logics'] are not changed in this process. Maybe the code should be refactored to make it clearer?
game_logics = {action_mapping[key]:{k:[t if t not in action_mapping else action_mapping[t] for t in game_logics[key][k]] for k in game_logics[key]} for key in game['game_logics']}
map = game['map']
print(game_logics)

story  = [key for key in game_logics] # every key is an action, which reflects the progress of the story. TODO: Are we sure the story is ordered?
story_expanded = [key.replace('.','')+' '+game_logics[key]['location'][0]+'.' for key in story] # Add location info. Example: ['adventure confront the werewolf pack leader with sword at werewolf den.']
rooms = ['at '+s.split(' at ')[-1].replace('.','') for s in story_expanded] # Extract room info. Example: ['werewolf den']
all_objects = []
all_rooms = [] # all rooms can come from two sources: the room info in the story, and the room info in the map
all_npcs = []
for key in map:
    all_rooms.append(key[3:]) # after removing the 'at ' prefix, the key is the room name
    for item in map[key]:
        if item['type']=='npc':
            all_npcs.append(item['content'][4:])
        elif item['type']=='item':
            all_objects.append(item['content'][4:])
all_rooms = list(set(all_rooms))
all_objects = list(set(all_objects))
all_npcs = list(set(all_npcs))
print(all_rooms)
print(all_objects)
print(all_npcs)


{'adventurer speak with the village elders.': {'item needed': [], 'location': ['at village hall'], 'preceeding_events': [], 'description': [], 'results': []}, 'adventurer read books.': {'item needed': [], 'location': ['at library'], 'preceeding_events': [], 'description': ['There is a book at the library.'], 'results': ['has book']}, 'adventurer find maps.': {'item needed': ['has books'], 'location': ['at library'], 'preceeding_events': [], 'description': ['There is a map at the library.'], 'results': ['has map']}, 'adventurer take torch.': {'item needed': ['has money'], 'location': ['at general store'], 'preceeding_events': [], 'description': ['There is a torch for sale at the general'], 'results': ['has torch']}, 'adventurer find the werewolf den with maps, torch': {'item needed': ['has maps', 'has torch'], 'location': ['at forest'], 'preceeding_events': ['adventurer speak with the village elders.'], 'description': [], 'results': []}, 'adventurer take basket.': {'item needed': [], 'l

In [ ]:
import traceback
#initialize game engine
#action_template = 'gathered {enum(object)} for {object1} with {object2}'
#parameters = {'enum(object)': 'ingredients', 'object1': 'a cure', 'object2': 'a basket'}
#parametric_fields_attributes = {'enum(object)': {'is_enum_object': 'True'}, 'object1': {'is_object': 'True', 'is_cure': 'True', 'is_complete': 'AnyValue'}, 'object2': {'is_object': 'True', 'is_container': 'True'}}
#effects = ['Move {enum(object)} to {object2}', 'Set {object1.is_complete} to {True}']
game['game_logics']=game_logics
#initialize game environment 
#
game_engine = GameEngine(map, game_logics) # The game engine the computer will interact with and evaluate on during game generation. The actions are executed, so we can know is the game is actually playable or not.
replay_game_engine = GameEngine(map, game_logics) # The game engine that the player will interact with. It is a copy of game_engine, and has all the information about the admissible actions. but the actions are not executed.
#
###################################################################################################################################

# for index in range(3):
for index in range(len(story)):
    print("################## %i of %i action in the story ####################\n"%(index,len(story)))
    # sentence already has the "with" clause. sentence_expanded also has the location info.
    # Example: sentence: adventure confront the werewolf pack leader with sword.
    # Example: sentence_expanded: adventure confront the werewolf pack leader with sword at werewolf den.
    sentence,sentence_expanded,room = story[index], story_expanded[index], rooms[index]
    print("sentence:",sentence)
    print("sentence_expanded:",sentence_expanded)
    print("room:",room)
    sucess = False
    cnt = 0

    #move money to adventurer's inventory since planner assumes several precondition to be true to end the recursion
    # TODO: We may remove this in the future.
    #try:
    #    game_engine.world.find_node("money")
    #except Exception:
    #    game_engine.items.append(Item("money","A "+"money"))
    #    game_engine.items_ptrs["money"]=game_engine.items[-1]
    #    game_engine.world.add_node(game_engine.items[-1],game_engine.player)
    #if game_engine.world.find_node("money").container!=game_engine.player:
    #    raise Exception("money is not in player's inventory")
        # MoveNodeOperation(game_engine.world, 'money', 'Player').apply()


    # add the action to the game, and check if the action is actually runnable
    previous_attempt_info = ""
    try:
        print("attempt %i"%cnt,sentence_expanded)
        sentence_fixed_grammar, annotated_sentence = fix_grammar_and_annotate_entities(sentence if sentence[-1]=='.' else sentence+'.', all_objects=all_objects, all_rooms = all_rooms, all_npcs=all_npcs)
        template,action_rule,attempt_info = get_action_template_and_rules(annotated_sentence,previous_attempt_info)
        previous_attempt_info = previous_attempt_info+'\n\n//The following is a wrong example that fail to execute. You should avoid generating the same content in your next trial. Refers to the error message below for more detail\n'+attempt_info
        print("debug #############\n",attempt_info,'#################')
        # interact with game engine and check if the action is actually runnable
        replay_game_engine_copy = copy.deepcopy(replay_game_engine)
        game_engine = try_adding_action(game_engine, sentence, sentence_fixed_grammar,sentence_expanded,room,template,action_rule,replay_game_engine_copy)
        sucess = True   
    except Exception as e:
        sucess = False
        if "'NoneType' object has no attribute 'group'" in str(e): # TODO: Handle exceptions locally.
            previous_attempt_info = '\n\n'.join(previous_attempt_info.split('\n\n')[:-1]) #  TODO: Just remove the last attempt info? Is it a good idea.
        else:
            previous_attempt_info = previous_attempt_info+'\n\nError: '+str(e) 
        print(e,traceback.format_exc())
    print(sucess)
    while not sucess:
        if cnt>7:
            raise ValueError('Adding action: max number of attempt exeeded.')
        if cnt%2==0 and cnt!=0:
            previous_attempt_info = ""
        print("###############\nattemp %i failed, try adding action again!"%cnt)

        # TODO: The following code is the same as above, should be refactored.
        try:
            cnt+=1
            print("attempt %i"%cnt,sentence_expanded)
            sentence_fixed_grammar, annotated_sentence = fix_grammar_and_annotate_entities(sentence if sentence[-1]=='.' else sentence+'.', all_objects=all_objects, all_rooms = all_rooms, all_npcs=all_npcs)
            
            template,action_rule,attempt_info = get_action_template_and_rules(annotated_sentence,previous_attempt_info+('\n\nYour last attempt failed to execute. Revise it based on the error message....' if previous_attempt_info!="" else ""))
            print("debug #############\n",attempt_info,'#################')
            previous_attempt_info = previous_attempt_info+'\n\n//The following is a wrong example that fail to execute. You should avoid generating the same content in your next trial. Refers to the error message below for more detail\n'+attempt_info
            
            # interact with game engine and check if the action is actually runnable
            replay_game_engine_copy = copy.deepcopy(replay_game_engine)
            game_engine = try_adding_action(game_engine, sentence ,sentence_fixed_grammar,sentence_expanded,room,template,action_rule,replay_game_engine_copy)
            sucess = True
        except Exception as e:
            sucess = False
            print(e,'\n',traceback.format_exc())
            if "'NoneType' object has no attribute 'group'" in str(e):
                previous_attempt_info = '\n\n'.join(previous_attempt_info.split('\n\n')[:-1])
            else:
                previous_attempt_info = previous_attempt_info+'\n\nError: '+str(e)
    
    # We have successfully added the action to the game engine. We have also ensured that the desired effect is achieved in try_adding_action.
    # replay_game_engine is the game engine that has all the information of the actions, but the actions are not actually executed. 
    # a real player will play the game with replay_game_engine.
    # Now that the system has successfully added the action to the game engine and replay_game_engine_copy, 
    # we are sure that replay_game_engine_copy is runnable. 
    # Now Copy the current state of replay_game_engine_copy to replay_game_engine.
    replay_game_engine = copy.deepcopy(replay_game_engine_copy) 
    # succeed mark objective as done.
    # This field is used when checking action preconditions.
    game_engine.action_state[sentence]=True
    # find primary parameter in the sentence. e.g. find a cave with torch and map -- primary parameter is cave template is find {object}
    if 'with' not in sentence:
        primary_parameter = parsing_parameters(sentence_fixed_grammar.split(' with ')[0].strip(),template.split(' with ')[0].strip()) # TODO: What about phrases like "play with the dog"?
    else:
        primary_parameter = parsing_parameters(sentence_fixed_grammar.strip(),template.strip())
    # get parameter alias. Eg: a name in parameter to a name that is actually used in the game. Usually the two are the same.
    primary_parameter = [get_alias(list(game_engine.items_ptrs)+list(game_engine.npcs_ptrs)+list(game_engine.locations_ptrs), primary_parameter[k]) for k in primary_parameter]
    #print({"sentence_fixed_grammar":sentence_fixed_grammar,"primary_parameter":primary_parameter,'template':template,'keywords':copy.deepcopy(primary_parameter), \
    #                                                 'target_action_in_game_logic': sentence})
    # store keywords, action pairs
    game_engine.keywords_to_game_logic_action_map.append({'template':template,'keywords':primary_parameter, \
                                                     'target_action_in_game_logic': sentence})
    for disired_effect in game_engine.game_logics[sentence]['results']:
        if disired_effect[:4]=='has ':
            game_engine.obtainable_item_to_game_logic_action_map[get_alias(game_engine.items_ptrs.keys(),disired_effect[4:])].append(sentence)

# finishing construct replay game engine from game engine (Reference 1)
game_engine.action_template = sorted(list(game_engine.action_template), key=lambda x: 0 if ' with ' in x else 1)
replay_game_engine.action_template = copy.deepcopy(game_engine.action_template)
replay_game_engine.action_effects = copy.deepcopy(game_engine.action_effects)
replay_game_engine.action_general_constraints = copy.deepcopy(game_engine.action_general_constraints)
replay_game_engine.keywords_to_game_logic_action_map = copy.deepcopy(game_engine.keywords_to_game_logic_action_map)
replay_game_engine.obtainable_item_to_game_logic_action_map = copy.deepcopy(game_engine.obtainable_item_to_game_logic_action_map)

add node: village hall
add node: village elders
laction on map (0, 0)
add node: forest
add node: werewolf
add node: farmer
add node: herbalist
add node: witch
add node: basket
add node: wise woman
laction on map (0, 1)
add node: general store
add node: torch
add node: lockpicking tool
add node: merchant
laction on map (0, 2)
add node: library
add node: map
add node: book
laction on map (0, 3)
add node: abandoned mansion
laction on map (0, 4)
add node: store
add node: flashlight
add node: merchant
laction on map (0, 5)
add node: werewolf den
add node: werewolf pack leader
laction on map (0, 6)
add node: weapon depot
add node: sword
laction on map (0, 7)
add node: thieves den
add node: key
laction on map (0, 8)
add node: village
add node: villager
laction on map (0, 9)
add node: armor shop
add node: armor
laction on map (0, 10)
add node: dungeon
add node: weapon
laction on map (0, 11)
{'basket': [Item]@(basket-d16fc0bb-f107-48a2-82a4-8a3a9cec1bfb), 'torch': [Item]@(torch-987f7618-e6c5-4e

In [ ]:
# run this cell to store replay_game_engine for testing
replay_game_engine_copy = copy.deepcopy(replay_game_engine)

In [ ]:
# run this cell to restore replay_game_engine for testing
replay_game_engine = copy.deepcopy(replay_game_engine_copy)

In [ ]:
# save game
replay_game_engine.save('replay_game_world_werewolf.json','replay_game_engine_werewolf.json')

In [ ]:
# load game
replay_game_engine.load('replay_game_world_werewolf.json','replay_game_engine_werewolf.json')

In [ ]:
replay_game_engine.keywords_to_game_logic_action_map

[{'template': 'speaks with {enum(npc)}',
  'keywords': ['village elders'],
  'target_action_in_game_logic': 'adventurer speak with the village elders.'},
 {'template': 'reads {enum(object)}',
  'keywords': ['book'],
  'target_action_in_game_logic': 'adventurer read books.'},
 {'template': 'finds {object1}',
  'keywords': ['map'],
  'target_action_in_game_logic': 'adventurer find maps.'}]

In [ ]:
def user_friendly_action_interpreter(sentence):
    # find the targeting action if the user input can be interpret as a step in game_logic
    # Note that users can either enter a sentence in game_logic or a sentence not in game_logic
    global replay_game_engine
    sentence_in_game_logic = ""
    for keywords_to_game_logic_action in replay_game_engine.keywords_to_game_logic_action_map:
        fieldnames_in_template = [name for _, name, _, _ in Formatter().parse(keywords_to_game_logic_action['template']) if name]
        p = str(keywords_to_game_logic_action['template'])
        for fieldname in fieldnames_in_template:
            p = p.replace('{'+fieldname+'}','(.*)')
        result = re.compile(p).search(sentence)
        if result:
            print("test find action in game_logics",keywords_to_game_logic_action['template'],keywords_to_game_logic_action['keywords'],sentence)
            if all([t in sentence for t in keywords_to_game_logic_action['keywords']]):
                sentence_in_game_logic = keywords_to_game_logic_action['target_action_in_game_logic']

    print("chosen sentence in game logic",sentence,sentence_in_game_logic)
    for template in replay_game_engine.action_template:

        #TODO: need to make this code block into a function
        #parse parameters
        print("template",template)
        fieldnames_in_template = [name for _, name, _, _ in Formatter().parse(template) if name]
        print(fieldnames_in_template)
        p = str(template)
        for fieldname in fieldnames_in_template:
            p = p.replace('{'+fieldname+'}','(.*)')
        print("matching pattern:",sentence,p)
        print(p)
        p = re.compile(p)
        result = p.search(sentence)
        if result:
            parameters = {}
            for i,fieldname in enumerate(fieldnames_in_template):
                if fieldname=='enum(npc)' or fieldname=='enum(object)':
                    parameters[fieldname] = [s.strip() for s in result.group(i+1).split(',') if s.strip()!='']
                else:
                    parameters[fieldname] = [result.group(i+1).replace('.','')]
            print("parameters",parameters)
            
            #TODO may add some fuzzy matching here

            #check preconditions
            if sentence_in_game_logic in replay_game_engine.game_logics:
                replay_game_engine.check_if_precondtions_have_been_met(sentence_in_game_logic)
                for entity in replay_game_engine.action_general_constraints[template]:
                    for attribute in replay_game_engine.action_general_constraints[template][entity]:
                        #print(entity,attribute, "should be equal to", replay_game_engine.action_general_constraints[template][entity][attribute])
                        for t in parameters[entity]:
                            if replay_game_engine.world.find_node(t)._get_attribute(attribute)!='AnyValue' and \
                                replay_game_engine.action_general_constraints[template][entity][attribute]!='AnyValue' and \
                                replay_game_engine.action_general_constraints[template][entity][attribute]!=replay_game_engine.world.find_node(t)._get_attribute(attribute):
                                    print("require", attribute, "to be",replay_game_engine.action_general_constraints[template][entity][attribute], \
                                          'got object',t, 'with attribute value',replay_game_engine.world.find_node(t)._get_attribute(attribute))
                                    return False
            
            # check if game_logic has sefecify ways to obtain an item. If yes, you can only obtain certain items with the methods defined in game_logics
            for effect in replay_game_engine.action_effects[template]:
                if 'Move ' in effect:
                    param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                    assert len(param)==2
                    param_object, param_to = param[0][1:-1],param[1][1:-1]
                    if param_object in parameters and param_to=='inventory' and \
                                    all([t in replay_game_engine.obtainable_item_to_game_logic_action_map for t in parameters[param_object]]):
                        replay_game_engine.obtainable_item_to_game_logic_action_map_ = copy.deepcopy(replay_game_engine.obtainable_item_to_game_logic_action_map)
                        for item in parameters[param_object]:
                            if sentence_in_game_logic in replay_game_engine.obtainable_item_to_game_logic_action_map[item]:
                                print("Congratulation: you hit the correct action to obtain",item)
                                replay_game_engine.obtainable_item_to_game_logic_action_map[item].\
                                    pop(replay_game_engine.obtainable_item_to_game_logic_action_map[item].index(sentence_in_game_logic))
                            else:
                                print("It doesn't work.")
                                replay_game_engine.obtainable_item_to_game_logic_action_map = copy.deepcopy(replay_game_engine.obtainable_item_to_game_logic_action_map_)
                                return False
                    

            # apply effects
            for effect in replay_game_engine.action_effects[template]:
                print(effect)
                if 'Move ' in effect:
                    param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                    assert len(param)==2
                    print(param)
                    param_object, param_to = param[0][1:-1],param[1][1:-1]
                    if param_object not in parameters:
                        raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                    if param_to=='inventory':
                        param_to = 'Player'
                    elif param_to== "environment":
                        param_to = replay_game_engine.world.player.container.name
                    elif '.inventory' in param[1]:
                        param_to= param_to.replace('.inventory','')
                    elif param_to in parameters:
                        param_to = parameters[param_to]
                        if isinstance(param_to, list):
                            param_to = param_to[0]
                    for t in parameters[param_object]:
                        print("Moved",t,param_to)
                        can_proceed_flag = False
                        temp = replay_game_engine.world.find_node(t)
                        while temp:
                            if temp==replay_game_engine.world.player.container:
                                can_proceed_flag = True
                                break
                            else:
                                temp = temp.container
                        if can_proceed_flag==False:
                            raise Exception("Execution Error: in %s. You are not allowed to move an entity if the entity is in a different room"%s)
                        replay_game_engine.world.move_node(replay_game_engine.world.find_node(t), replay_game_engine.world.find_node(param_to))               
                        #MoveNodeOperation(replay_game_engine.world, t, param_to).apply()
                elif "Add " in effect:
                    param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                    assert len(param)==1
                    param_object = param[0][1:-1]
                    if param_object not in parameters:
                        raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                    for t in parameters[param_object]:
                        print("Added",t)
                        if 'npc' in param:
                            replay_game_engine.world.add_node(Character(t,'A '+t), replay_game_engine.world.find_node("Player").container)  
                        if 'object' in param:
                            replay_game_engine.world.add_node(Item(t,'A '+t), replay_game_engine.world.find_node("Player").container)               
                        #AddNodeOperation(replay_game_engine.world, t, Item, "A "+t, replay_game_engine.world.player.container.name).apply()
                elif "Delete " in effect:
                    param = re.findall(r'\{[a-z1-9 \(\)]+\}', effect)
                    assert len(param)==1
                    param_object = param[0][1:-1]
                    if param_object not in parameters:
                        raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                    for t in parameters[param_object]:
                        print("Deleted",t)
                        replay_game_engine.world.remove_node(replay_game_engine.world.find_node(t))
                        #DeleteNodeOperation(replay_game_engine.world, t).apply()

                elif "Set " in effect:
                    param = re.findall(r'\{[a-zA-Z1-9._ \(\)]+\}', effect)
                    assert len(param)==2
                    attribute, value = param[0][1:-1], param[1][1:-1]
                    assert '.' in attribute
                    assert value in ['True','False','AnyValue']
                    param_object = attribute.split('.')[0]
                    attribute_name = attribute.split('.')[1]
                    if param_object not in parameters:
                        raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()])))
                    for t in parameters[param_object]:
                        print("Set Attribute",t,attribute_name, value)
                        replay_game_engine.world.set_node_attribute(replay_game_engine.world.find_node(t),attribute_name, value)
                        #SetNodeAttributeOperation(replay_game_engine.world, t, attribute_name, value).apply()
                elif "Display " in effect:
                    param = re.findall(r'\{[a-zA-Z1-9._ \(\)]+\}', effect)
                    assert len(param)==1
                    attribute = param[0][1:-1]
                    param_object = attribute.split('.')[0]
                    if param_object not in parameters and param_object not in ['inventory','environment']:
                        raise Exception("%s not defined. Valid parameters: %s"%(param_object,', '.join([k for k in parameters.keys()]+['inventory','environment'])))
                    print("Displaying Message--- Detial:",effect)
                    #TODO implement DisplayMessageOperation and fill this
            replay_game_engine.action_state[sentence_in_game_logic]=True
            return True
            
    print("not an admissible action")

## play the game manualy following the gold story

In [ ]:
# load game
replay_game_engine.load('replay_game_world_werewolf.json','replay_game_engine_werewolf.json')

In [ ]:
replay_game_engine.action_template

['speaks with {enum(npc)}', 'reads {enum(object)}', 'finds {object1}']

In [ ]:
print(user_friendly_action_interpreter("speaks with village elders"))

test find action in game_logics speaks with {enum(npc)} ['village elders'] speaks with village elders
chosen sentence in game logic speaks with village elders adventurer speak with the village elders.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: speaks with village elders speaks with (.*)
speaks with (.*)
parameters {'enum(npc)': ['village elders']}
True


In [ ]:
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("library"))
#MoveNodeOperation(replay_game_engine.world, "Player", "library")
print(user_friendly_action_interpreter("reads book"))

test find action in game_logics reads {enum(object)} ['book'] reads book
chosen sentence in game logic reads book adventurer read books.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: reads book speaks with (.*)
speaks with (.*)
template reads {enum(object)}
['enum(object)']
matching pattern: reads book reads (.*)
reads (.*)
parameters {'enum(object)': ['book']}
Congratulation: you hit the correct action to obtain book
Move {enum(object)} to {inventory}
['{enum(object)}', '{inventory}']
Moved book Player
Display {enum(object).message}
Displaying Message--- Detial: Display {enum(object).message}
True


In [ ]:
print(replay_game_engine.world.find_node("book").container)
print(replay_game_engine.world.find_node("Player").container)

[Player]@(Your goal is resolve the curse of the werewolf)
[Room]@(library-9697a694-d0af-4f83-8743-4a805d92a014)


In [ ]:
#still at library
user_friendly_action_interpreter("finds map")

test find action in game_logics finds {object1} ['map'] finds map
chosen sentence in game logic finds map adventurer find maps.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: finds map speaks with (.*)
speaks with (.*)
template reads {enum(object)}
['enum(object)']
matching pattern: finds map reads (.*)
reads (.*)
template finds {object1}
['object1']
matching pattern: finds map finds (.*)
finds (.*)
parameters {'object1': ['map']}
Congratulation: you hit the correct action to obtain map
Move {object1} to {inventory}
['{object1}', '{inventory}']
Moved map Player
Display {object1.message}
Displaying Message--- Detial: Display {object1.message}


True

# Let's try if we can get the map without getting books first

In [ ]:
# load game
replay_game_engine.load('replay_game_world_werewolf.json','replay_game_engine_werewolf.json')

print(user_friendly_action_interpreter("speaks with village elders"))
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("library"))
#MoveNodeOperation(replay_game_engine.world, "Player", "library")
print(user_friendly_action_interpreter("finds map"))


test find action in game_logics speaks with {enum(npc)} ['village elders'] speaks with village elders
chosen sentence in game logic speaks with village elders adventurer speak with the village elders.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: speaks with village elders speaks with (.*)
speaks with (.*)
parameters {'enum(npc)': ['village elders']}
True
test find action in game_logics finds {object1} ['map'] finds map
chosen sentence in game logic finds map adventurer find maps.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: finds map speaks with (.*)
speaks with (.*)
template reads {enum(object)}
['enum(object)']
matching pattern: finds map reads (.*)
reads (.*)
template finds {object1}
['object1']
matching pattern: finds map finds (.*)
finds (.*)
parameters {'object1': ['map']}


Exception: ignored

# Now let try if we cab bypass some preceeding events (obtain the map without finds map)

In [ ]:
replay_game_engine.obtainable_item_to_game_logic_action_map

{'book': ['adventurer read books.'], 'map': ['adventurer find maps.']}

### Note that read {enum(object)} in this game move the objects into inventory

In [ ]:
print(replay_game_engine.action_effects['reads {enum(object)}'])

['Move {enum(object)} to {inventory}', 'Display {enum(object).message}']


In [ ]:
# load game
replay_game_engine.load('replay_game_world_werewolf.json','replay_game_engine_werewolf.json')

print(user_friendly_action_interpreter("speaks with village elders"))
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("library"))
#MoveNodeOperation(replay_game_engine.world, "Player", "library")
print(user_friendly_action_interpreter("reads map"))


test find action in game_logics speaks with {enum(npc)} ['village elders'] speaks with village elders
chosen sentence in game logic speaks with village elders adventurer speak with the village elders.
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: speaks with village elders speaks with (.*)
speaks with (.*)
parameters {'enum(npc)': ['village elders']}
True
test find action in game_logics reads {enum(object)} ['book'] reads map
chosen sentence in game logic reads map 
template speaks with {enum(npc)}
['enum(npc)']
matching pattern: reads map speaks with (.*)
speaks with (.*)
template reads {enum(object)}
['enum(object)']
matching pattern: reads map reads (.*)
reads (.*)
parameters {'enum(object)': ['map']}
It doesn't work.
False


# not included in this test

In [ ]:
print(replay_game_engine.world.find_node("map").container)

[Player]@(Your goal is resolve the curse of the werewolf)


In [ ]:
#move to general store
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("general store"))
user_friendly_action_interpreter("took torch", 'adventurer take torch.')


took torch adventurer take torch.
spoke with (.*)
reads (.*)
found (.*)
took (.*)
parameters {'object1': ['torch']}
Move {object1} to {inventory}
['{object1}', '{inventory}']
Moved torch Player


True

In [ ]:
print(replay_game_engine.world.find_node("torch").container)
print(replay_game_engine.world.find_node("Player").container)

Player(player-856411f5-5e7b-4dee-bf4e-fda3cde80058, Player, You are the player.)
Room(general-store-9dded7e9-c813-4e23-b32f-6bc63f9b2ebe, general store, You are at general store)


In [ ]:
#move to general forest
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("forest"))
user_friendly_action_interpreter("found werewolf den with map, torch", 'adventurer find the werewolf den with maps, torch')


found werewolf den with map, torch adventurer find the werewolf den with maps, torch
spoke with (.*)
reads (.*)
found (.*)
parameters {'object1': ['werewolf den with map, torch']}


ValueError: ignored

In [ ]:
#still at forest
user_friendly_action_interpreter("took basket", 'adventurer take basket.')


took basket adventurer take basket.
spoke with (.*)
reads (.*)
found (.*) within (.*)
took (.*)
parameters {'object1': ['basket']}
Move {object1} to {inventory}
['{object1}', '{inventory}']
Moved basket Player


True

In [ ]:
print(replay_game_engine.world.find_node("basket").container)

Player(player-d68160f3-ef31-4f16-a31b-943c98d1b3b0, Player, You are the player.)


## Test action not in the gold story

In [ ]:
replay_game_engine = copy.deepcopy(replay_game_engine_copy)
replay_game_engine.action_template = copy.deepcopy(game_engine.action_template)
replay_game_engine.action_effects = copy.deepcopy(game_engine.action_effects)
replay_game_engine.action_general_constraints = copy.deepcopy(game_engine.action_general_constraints)
print(replay_game_engine.action_state)


defaultdict(<function GameEngine.__init__.<locals>.<lambda> at 0x7f4f162c4b80>, {})


In [ ]:
#test if I can go the library and take the book
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("library"))
user_friendly_action_interpreter("took book", '')


took book 
spoke with (.*)
reads (.*)
found (.*) within (.*)
took (.*)
parameters {'object1': ['book']}
Move {object1} to {inventory}
['{object1}', '{inventory}']
Moved book Player


True

In [ ]:
replay_game_engine.world.find_node("book").container

Player(player-d68160f3-ef31-4f16-a31b-943c98d1b3b0, Player, You are the player.)

In [ ]:
#move to general forest
replay_game_engine.world.move_node(replay_game_engine.world.find_node("Player"), replay_game_engine.world.find_node("forest"))
user_friendly_action_interpreter("found werewolf den with map, torch", 'adventurer find the werewolf den with maps, torch')


found werewolf den with map, torch adventurer find the werewolf den with maps, torch
spoke with (.*)
reads (.*)
found (.*) within (.*)
took (.*)
found (.*) with (.*)
parameters {'room1': ['werewolf den'], 'enum(object)': ['map', 'torch']}


Exception: ignored

#### Potential bugs

In [ ]:
#the high level wrapper seems to be buggy for the replay_game_engine, works fine for game_engine. It doesn't throw error either
print(replay_game_engine.world.find_node("book").container)
MoveNodeOperation(replay_game_engine.world, "book", "village hall")
print(replay_game_engine.world.find_node("book").container)

Room(library-be6c279a-6134-4345-99db-851e42943fbb, library, You are at library)
Room(library-be6c279a-6134-4345-99db-851e42943fbb, library, You are at library)


in game replay if two template start with the same prefix. It matches to the first template which may be wrong

e.g. template list defaultdict(int,
            {'spoke with {enum(npc)}': 1,
             'reads {enum(object)}': 2,
             'found {object1}': 3,
             'took {object1}': 4,
             'found {room1} with {enum(object)}': 5})

target action: "found werewolf den with map, torch" will match 'found {object1}' 

## TODO

export the game to json file.

    implement save/load for every subclass 